In [1]:
from langchain.llms import GooglePalm

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

api_key = os.getenv('API_KEY')

In [3]:
# print(api_key)

In [5]:
import google.generativeai as palm
llm = GooglePalm(google_api_key=api_key, temperature=0.5)

In [6]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path = 'codebasics_faqs.csv', source_column= 'prompt')
data = loader.load()

In [7]:
# data 

In [9]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS 

instructor_embeddings = HuggingFaceInstructEmbeddings()
vectorDB = FAISS.from_documents(documents = data, embedding = instructor_embeddings)


load INSTRUCTOR_Transformer
max_seq_length  512


In [12]:
retriever = vectorDB.as_retriever()
rdocs = retriever.get_relevant_documents("for how long is this course valid?")

In [13]:
rdocs 

[Document(page_content='prompt: Once purchased, is this course available for lifetime access?\nresponse: Yes', metadata={'source': 'Once purchased, is this course available for lifetime access?', 'row': 22}),
 Document(page_content='prompt: What is the duration of this bootcamp? How long will it last?\nresponse: You can complete all courses in 3 months if you dedicate 2-3 hours per day.', metadata={'source': 'What is the duration of this bootcamp? How long will it last?', 'row': 8}),
 Document(page_content='prompt: Will the course be upgraded when there are new features in Power BI?\nresponse: Yes, the course will be upgraded periodically based on the new features in Power BI, and learners who have already bought this course will have free access to the upgrades.', metadata={'source': 'Will the course be upgraded when there are new features in Power BI?', 'row': 27}),
 Document(page_content='prompt: Is this bootcamp enough for me in Microsoft Power BI and\n Excel certifications?\nrespo

In [16]:
# to avoid hallucinations use this template

from langchain.prompts import PromptTemplate

prompt_template = """ Given the following text and a question, generate an answer based on this context only to provide as 
    much text as possible from "response" section in the source document context without making any changes. If the answer 
    is not found in the context, kindly state "I don't know". Do not make up answers.

CONTEXT = {context}

QUESTION = {question}
"""

In [17]:
PROMPT = PromptTemplate(
    template = prompt_template,
    input_variables = {"context", "question"},
)

In [18]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    input_key = "query",
    return_source_documents = True,
    chain_type_kwargs = {"prompt" : PROMPT}
)

In [19]:
chain("Do you have javscript course")

c:\Users\chira\OneDrive\Desktop\Frontend templates\LangChain\CodebasicsChatBot\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'Do you have javscript course',
 'result': "I don't know",
 'source_documents': [Document(page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}),
  Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}),
  D